# __Information Assurance__

## __Evolving trends in Information Assurance, a NLP analysis of the Literature from 1967 to 2024__

In this work, we will carry out a Systematic Topic Review (STR) for topic extraction and Chain of Density (CoD) + Few-Shots for summarizing the contents.

The ultimate goal of this work is to perform sumatizations for each decade from 1967 to the present to understand what has been researched in matters of information assurance.

## <font color='blue'>__Systematic Topic Review Modeling with MistralAI__</font>




1. **bertopic**: Bertopic is a library used for topic modeling with BERT-based language models. It allows for analyzing and clustering text documents into related topics, which is useful for natural language processing and text mining tasks.

2. **accelerate**: Accelerate is a library designed to speed up and optimize the training of machine learning models in PyTorch, especially in distributed or GPU configurations. It helps improve computational efficiency during model training.

3. **bitsandbytes**: Bitsandbytes is a package or project aimed at making Large Language Models (LLMs) more accessible by implementing 4-bit quantization and QLoRA techniques. These techniques reduce the computational and memory requirements associated with LLMs, facilitating their use in resource-limited applications and devices.

4. **xformers**: xFormers is a set of customizable building blocks that can be used without repetitive coding. These components are independent and customizable, making them suitable for various applications, such as computer vision and natural language processing (NLP). xFormers focuses on research and contains cutting-edge components not yet available in conventional libraries like PyTorch. It is also designed for efficiency in terms of iteration speed and memory usage, including its own CUDA kernels and the ability to use other relevant libraries as needed.

5. **adjustText**: adjustText is a small library designed to help you adjust text positions in matplotlib charts to eliminate or minimize overlaps between text and data points. This approach is based on the overlaps of text bounding boxes and iteratively moves them to reduce overlaps.

# 📄 **Preamble**

In [ ]:
%%capture
!pip install bertopic==0.15.0 accelerate==0.23.0 bitsandbytes==0.41.1 xformers==0.0.22 adjustText huggingface_hub==0.19.4 openai

In [ ]:
%%capture
!pip install transformers==4.36.1

Keep in mind that transformers, by design, include several stochastic components such as weight initialization, dropout layers, and the sequencing and batching of training. This could result in different outputs each time the model is run.



## **Data**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import pickle

# Load the pickle file
path_file = '/content/drive/MyDrive/_RESEACH/Information_assurance/Data/'
with open(path_file + 'df_scopus_info_assurance_v2.pkl', "rb") as f:
    dataset = pd.read_pickle(f)

In [ ]:
dataset.columns

Index(['Authors', 'Author full names', 'Author(s) ID', 'Title', 'Year',
       'Cited by', 'DOI', 'Link', 'Abstract', 'Author Keywords',
       'Index Keywords', 'Document Type', 'Source'],
      dtype='object')

In [ ]:
len(dataset)

62344

In [ ]:
dataset = dataset[dataset['Abstract'] != '[No abstract available]'].reset_index(drop=True)


In [ ]:
len(dataset)

60793

In [ ]:

abstracts = dataset[dataset['Year'].between(1990, 1999)]["Abstract"].to_list()
titles = dataset[dataset['Year'].between(1990, 1999)]["Title"].to_list()

In [ ]:
len(abstracts)

412

In [ ]:
abstracts = dataset["Abstract"].to_list()
titles = dataset["Title"].to_list()

Titulo y Abstract, ejemplo

In [ ]:
print(titles[8])
print(abstracts[8])

INFORMATION PROTECTION.
The subject of information security is discussed and several methods for protection of information are reviewed. Emphasis is placed on security methods to ensure that their application is consistent with management policy and intent.


## Login HuggingFace

In [ ]:
#hugging face token
token_hf="here your token"
from huggingface_hub import login
login(token_hf) #, add_to_git_credential=True)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


**Mistral**

In [ ]:
from torch import cuda
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

print(device)

cuda:0


## **Optimization & Quantization**

To load our model, we'll need to implement some optimization tricks. Given that we have limited VRAM and not an A100 GPU, we will need to "condense" the model a bit so that we can run it.

There are several tricks we can use, but the main one will be 4-bit quantization.

This process reduces the 64-bit representation to just 4 bits, which decreases the GPU memory we will need. It's a recent and quite elegant technique for efficiently loading and using massive language models. You can learn more about this method [here](https://arxiv.org/pdf/2305.14314.pdf) in the QLoRA paper and on the amazing HuggingFace blog [here](https://huggingface.co/blog/4bit-transformers-bitsandbytes).

In [ ]:
from torch import bfloat16
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

These four parameters we just executed are incredibly important and offer many applications of massive language models (LLM) to consumers:

* `load_in_4bit`
  * This allows us to load the model with 4-bit precision compared to the original 32-bit precision.
  * This provides us with an incredible speed boost and reduces the required memory.

* `bnb_4bit_quant_type`
  * This is the type of 4-bit precision. The article recommends using normalized floating 4-bit, so that's what we're going to use.

* `bnb_4bit_use_double_quant`
  * This is an interesting trick as it performs a second quantization after the first, further reducing the amount of bits needed.

* `bnb_4bit_compute_dtype`
  * The compute type used during computation, which further speeds up the model.



Using this configuration, we can start loading the model and the tokenizer:

In [ ]:
import torch
if torch.cuda.is_available():

    print(f"GPUs available: {torch.cuda.device_count()}")
    print(f"GPU device name: {torch.cuda.get_device_name(0)}")
    print(f"GPU VRAM: {torch.cuda.get_device_properties(0).total_memory / (1024 ** 2):.2f} MB")
else:
    print("No GPU found.")


In [ ]:
from huggingface_hub import HfApi

def check_model_availability(model_id):
    api = HfApi()
    try:
        model_info = api.model_info(model_id)
        return f"Model '{model_id}' is available on Hugging Face."
    except Exception as e:
        return f"Model '{model_id}' is not available on Hugging Face. Error: {e}"

model_id = "upstage/SOLAR-10.7B-Instruct-v1.0"
model_id = 'mistralai/Mistral-7B-Instruct-v0.1'
print(check_model_availability(model_id))


Model 'mistralai/Mistral-7B-Instruct-v0.1' is available on Hugging Face.


In [ ]:
# Llama 2 Tokenizer
from torch import bfloat16
import transformers


tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# Mistral Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map=None #'auto'
)
model.eval()

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
   

Using the model and tokenizer, we will create a HuggingFace transformers pipeline that will allow us to easily generate new text:

Here are some of the specific parameters being used in the provided code:

- **task='text-generation'**: This setting indicates that the model will be used for the task of text generation. In other words, the model is expected to generate text by continuing from a given input.
- **do_sample=True**: This parameter indicates that random sampling will be used to generate text. Instead of always generating the most likely output, the model will randomly select from the available options, which can lead to more diversified results.
- **temperature=0.1**: Temperature is a parameter that controls the randomness in the generation process. A low value like 0.1 makes the generation more deterministic and tends to produce more coherent results.
- **max_new_tokens=500**: This limits the maximum number of tokens (words or sub-units) that will be generated in the output. In this case, the generation is limited to a maximum of 500 tokens.
- **repetition_penalty=1.1**: Repetition penalty is a value used to penalize excessive repetition of certain tokens in generation. A value greater than 1.0, such as 1.1, slightly penalizes repetitions in the generation.

In [ ]:
# Our text generator
generator = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    #do_sample=True,
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1,
)

In [ ]:
model.generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}

## **Prompt Engineering**

Although we can give direct instructions to the model, there's actually a template we need to follow. The template looks like this:

```python
"""
<s>[INST] <<SYS>>

{{ System Prompt }}

<</SYS>>

{{ User Prompt }} [/INST]

{{ Model Answer }}
"""
```

EThis template consists of two main components, namely the `{{ System Prompt }}` and the `{{ User Prompt }}`:
* The `{{ System Prompt }}` helps us guide the model during a conversation. For example, we can state that it is a helpful assistant specialized in labeling topics.
* The `{{ User Prompt }}` is where we ask a question.

You may have noticed the `[INST]` tags, which are used to identify the start and end of an instruction. We can use them to model the conversation history, as we will see in more detail later.

Next, let's see how we can use this template to optimize Mistral for topic modeling.

### **Prompt Template**

Our `system prompt` simple and straightforward :

In [ ]:
# System prompt describes information given to all conversations
system_prompt = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for labeling topics.
<</SYS>>
"""

We will tell the model that it is simply a helpful assistant for labeling topics, as that is our main objective.

On the other hand, our `user prompt` will be a bit more comprehensive. It will consist of two components: an **example** and the **main prompt**.

Let's start with the **example**. Most LLMs perform much better at generating accurate responses if you provide them with an example to work from. We will show an exact example of the type of result we expect.

In [ ]:
# Example prompt demonstrating the output we are looking for
example_prompt = """
I have a topic that contains the following documents:
- Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
- Meat, but especially beef, is the word food in terms of emissions.
- Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

The topic is described by the following keywords: 'meat, beef, eat, eating, emissions, steak, food, health, processed, chicken'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.

[/INST] Environmental impacts of eating meat
"""

This example, based on a series of keywords and documents primarily about the impact of meat, helps the model understand the type of result it should provide. We show the model that we expect only the label, which is easier for us to extract.

Next, we will create a template that we can use within BERTopic:

In [ ]:
# Our main prompt with documents ([DOCUMENTS]) and keywords ([KEYWORDS]) tags
main_prompt = """
[INST]
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.
[/INST]
"""

There are two specific BERTopic tags of interest, namely `[DOCUMENTS]` and `[KEYWORDS]`:

* `[DOCUMENTS]` contains the 5 most relevant documents for the topic.
* `[KEYWORDS]` contains the 10 most relevant keywords for the topic generated through c-TF-IDF.

This template will be filled in according to each topic. And finally, we can combine this into our final prompt:

In [ ]:
prompt = system_prompt + example_prompt + main_prompt

## **BERTopic**

Before we begin with topic modeling, we must first perform two steps:
* Precompute embeddings.
* Define submodels.

By precomputing embeddings for each document, we can accelerate further exploration steps and use the embeddings to quickly iterate over BERTopic's hyperparameters if necessary.

In [ ]:
abstracts[:3]

['In retail and international banking, there is a growing demand for upto-date information. Security is an important factor in all aspects of bank work. © 1985.',
 'In the future, the security of computer and data communications systems will to a large extent depend on the secure handling of secret parameters. Such parameters include log-on passwords, Personal Identification Numbers(PINs), encryption keys, encryption Initialization Vectors (IVs), pseudo-random number-generator quantities, and biometric personal identification quantities like digitized fingerprints. The greater dependency on such security parameters will be the natural outgrowth of both the increasing sophistication and the increasing computerization of security functions. A new approach to protected storage and protected processing of security parameters is the use of a special type of hardware/firmware-based equipment variously referred to as security modules, tamper-resistant modules, and cryptographic security modul

In [ ]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("BAAI/bge-large-en-v1.5") # Postion 14 in Leaderboar. Embedding dimension = 1024, max imput tokens = 512
# The model is transformed into an embedding using the previous transformer.
embeddings = embedding_model.encode(abstracts, show_progress_bar=True)

Batches:   0%|          | 0/1900 [00:00<?, ?it/s]

## **Sub-models**

Next, we will define all the submodels in BERTopic and make some minor adjustments to the number of clusters to be created, setting random_state, etc.

__UMAP Parameters__

- __n_neighbors__: This parameter controls the size of the local neighborhood from which each point draws information to be projected into a low-dimensional space. A higher value considers a broader context, making the global structure more prominent in the low-dimensional representation. A lower value causes the projection to focus more on the local structure, potentially revealing clusters or local patterns at the expense of losing the overall view of the global structure.

- __n_components__: Specifies the number of dimensions of the projected space (i.e., the low-dimensional space to which the data will be reduced). For example, n_components=2 will project the data onto a two-dimensional plane, which is useful for visualizations. Increasing n_components allows capturing more details of the data structure in more dimensions, but it can make visualizing and interpreting the results more challenging.



In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN

# Dimensionality reduction
umap_model = UMAP(n_neighbors=10, n_components=6, min_dist=0.0, metric='cosine', random_state=42) # before n_neighbors = 10
# Clustering - Hierarchical Density-Based Spatial Clustering of Applications with Noise
hdbscan_model = HDBSCAN(min_cluster_size=3, metric='euclidean', cluster_selection_method='eom', prediction_data=True) # before min_cluster_size=120

As a small additional benefit, we will reduce the embeddings we created earlier to 2 dimensions so that we can use them for visualization purposes once we have created our topics.

In [ ]:
# Pre-reduce embeddings for visualization purposes
reduced_embeddings = UMAP(n_neighbors=10, n_components=3, min_dist=0.0, metric='cosine').fit_transform(embeddings) # before n_neighbors = 15 or 4

In [ ]:
import numpy as np


# Adding noise to the embeddings
noise_strength = 1e-4
noisy_embeddings = embeddings + np.random.normal(0, noise_strength, embeddings.shape)

# UMAP execution with noisy embeddings
reduced_embeddings = UMAP(n_neighbors=4, n_components=2, min_dist=0.0, metric='cosine').fit_transform(noisy_embeddings)


### **Representation Models**

One of the ways we are going to represent the topics is with SOLAR, which should provide us with a clear label. However, we may want to have additional representations to view a topic from multiple perspectives.

Here, we are going to use c-TF-IDF as our main representation and [KeyBERT](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html#keybertinspired), [MMR](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html#maximalmarginalrelevance), and [LLM](https://maartengr.github.io/BERTopic/getting_started/representation/llm.html) as our additional representations.

Three different representation models are used because each provides a unique perspective and can be useful for different tasks and objectives in topic modeling. Here is a brief explanation of why three models are used:

1. **KeyBERTInspired (KeyBERT)**: This model is based on the idea of extracting keywords from documents. It provides a keyword-based representation that highlights the most important words in a document.

2. **MaximalMarginalRelevance (MMR)**: MMR focuses on maximizing the relevance of selected documents while maintaining adequate diversity. It allows selecting documents that are relevant but also diverse, which can be useful to avoid redundancy in the results of the topic clustering.

3. **Text Generation with Mistral**: Mistral is a text generator that can be used to generate longer text representations or to expand topic descriptions. It provides a more generative perspective that can be useful for creating summaries or expanding topics in detail.

In [ ]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration

# KeyBERT
keybert = KeyBERTInspired()

# MMR
mmr = MaximalMarginalRelevance(diversity=0.3)

# Text generation with SOLAR
llmodel = TextGeneration(generator, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    "Mistral": llmodel,
    "MMR": mmr,
}

## **Training**

Now that we have our models prepared, we can start training our topic model! We supply BERTopic with the submodels of interest, run `.fit_transform`, and observe what kinds of topics we get.

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Sub-models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(abstracts, embeddings)


Streaming output truncated to the last 5000 lines.
100%|██████████| 5695/5695 [1:35:20<00:00,  1.00s/it]


Now that we have finished training our model, let's see what topics were generated:


BerTopic documentation:

https://maartengr.github.io/BERTopic/getting_started/quickstart/quickstart.html#quick-start

In [ ]:
len(abstracts)

78

In [ ]:
# Show topics
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,Mistral,MMR,Representative_Docs
0,-1,13,-1_and_the_information_security,"[and, the, information, security, of, in, are,...","[systems, communications, technology, system, ...","[Information systems and security, , , , , , ,...","[and, the, information, security, of, in, are,...",[The following topics are dealt with: network ...
1,0,20,0_the_and_of_information,"[the, and, of, information, security, to, is, ...","[security, management, systems, technical, sys...","[Information Security Assessment Methods, , , ...","[the, and, of, information, security, to, is, ...",[Even if an organization has the best technica...
2,1,19,1_the_of_and_information,"[the, of, and, information, security, in, is, ...","[hackers, security, technical, industry, syste...","[Information Security and Local Area Networks,...","[the, of, and, information, security, in, is, ...",[Local area networks which handle many signals...
3,2,9,2_the_of_and_to,"[the, of, and, to, encryption, security, key, ...","[encryption, security, decryption, secure, sys...","[Encryption and Security in Computer Systems, ...","[the, of, and, to, encryption, security, key, ...",[This paper presents three important - and lik...
4,3,8,3_the_of_to_methods,"[the, of, to, methods, and, are, is, flow, con...","[cryptographic, security, acm, proofs, impleme...",[Information flow analysis in database managem...,"[the, of, to, methods, and, are, is, flow, con...",[A new approach to information flow in sequent...
5,4,5,4_the_system_security_infosec,"[the, system, security, infosec, of, and, adva...","[avionics, avionic, infosec, comsec, systems, ...","[Advanced avionics and information security, ,...","[the, system, security, infosec, of, and, adva...",[The authors identify INFOSEC (information sec...
6,5,4,5_the_and_to_of,"[the, and, to, of, oversight, classification, ...","[overclassification, oversight, security, clas...",[Information security classification and overs...,"[the, and, to, of, oversight, classification, ...",[This explanation of the development of E.O. 1...


In [ ]:
topic_distr, _ = topic_model.approximate_distribution(abstracts, min_similarity=0)
topic_model.visualize_distribution(topic_distr[15], custom_labels= True, width=1200)

100%|██████████| 1/1 [00:00<00:00, 20.77it/s]


In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.get_topic(1, full=True)["KeyBERT"] # Old run

[('hackers', 0.69429886),
 ('security', 0.6846518),
 ('technical', 0.6542234),
 ('industry', 0.65417784),
 ('systems', 0.6500517),
 ('electronic', 0.64637405),
 ('system', 0.6408932),
 ('network', 0.635321),
 ('communications', 0.62632096),
 ('technology', 0.6257345)]

We got over 100 topics that were created and they all seem quite diverse.We can use the labels by Llama 2 and assign them to topics that we have created. Normally, the default topic representation would be c-TF-IDF, but we will focus on Llama 2 representations instead.


In [ ]:
labels = [label[0][0].split("\n")[0] for label in topic_model.get_topics(full=True)["Mistral"].values()]
topic_model.set_topic_labels(labels)

In [ ]:
labels # New

['Information systems and security',
 'Information Security Assessment Methods',
 'Information Security and Local Area Networks',
 'Encryption and Security in Computer Systems',
 'Information flow analysis in database management systems',
 'Advanced avionics and information security',
 'Information security classification and oversight']

## **Visualize**
We can go through each topic manually, which would take a lot of work, or we can visualize them all in a single interactive graph.
BERTopic has a bunch of [visualization functions](https://medium.com/r/?url=https%3A%2F%2Fmaartengr.github.io%2FBERTopic%2Fgetting_started%2Fvisualization%2Fvisualize_documents.html) that we can use. For now, we are sticking with visualizing the documents.

In [ ]:
# topic_model.visualize_documents(abstracts, reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=False, custom_labels=True)

In [ ]:
topic_model.visualize_documents(titles, reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=False, custom_labels=True)

In [ ]:
visualization = topic_model.visualize_documents(titles, reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=False, custom_labels=True)

## **Advanced Visualization**

Although we can use the built-in visualization features of BERTopic, we can also create a static visualization that might be a bit more informative.

We start by creating the necessary variables that contain our reduced embeddings and representations:

In [ ]:
import itertools
import pandas as pd

# Define colors for the visualization to iterate over
colors = itertools.cycle(['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000'])
color_key = {str(topic): next(colors) for topic in set(topic_model.topics_) if topic != -1}

# Prepare dataframe and ignore outliers
df = pd.DataFrame({"x": reduced_embeddings[:, 0], "y": reduced_embeddings[:, 1], "Topic": [str(t) for t in topic_model.topics_]})
df["Length"] = [len(doc) for doc in abstracts]
df = df.loc[df.Topic != "-1"]
df = df.loc[(df.y > -10) & (df.y < 10) & (df.x < 10) & (df.x > -10), :]
df["Topic"] = df["Topic"].astype("category")

# Get centroids of clusters
mean_df = df.groupby("Topic").mean().reset_index()
mean_df.Topic = mean_df.Topic.astype(int)
mean_df = mean_df.sort_values("Topic")

In [ ]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(abstracts, calculate_tokens=True)

# Visualize the token-level distributions
distribucion = topic_model.visualize_approximate_distribution(abstracts[1], topic_token_distr[1])
distribucion

100%|██████████| 1/1 [00:00<00:00, 11.70it/s]


,In,the,future,the,security,of,computer,and,data,communications,systems,will,to,large,extent,depend,on,the,secure,handling,of,secret,parameters,Such,parameters,include,log,on,passwords,Personal,Identification,Numbers,PINs,encryption,keys,encryption,Initialization,Vectors,IVs,pseudo,random,number,generator,quantities,and,biometric,personal,identification,quantities,like,digitized,fingerprints,The,greater,dependency,on,such,security,parameters,will,be,the,natural,outgrowth,of,both,the,increasing,sophistication,and,the,increasing,computerization,of,security,functions,new,approach,to,protected,storage,and,protected,processing,of,security,parameters,is,the,use,of,special,type,of,hardware,firmware,based,equipment,variously,referred,to,as,security,modules,tamper,resistant,modules,and,cryptographic,security,modules,This,paper,describes,representative,functions,that,security,modules,perform,the,benefits,to,be,derived,when,security,modules,are,deployed,and,some,current,and,future,applications,of,this,new,technology,1986
0_the_and_of_information,0.174,0.369,0.577,0.927,1.110,1.162,1.102,0.913,0.662,0.560,0.543,0.382,0.276,0.132,0.000,0.000,0.000,0.101,0.101,0.101,0.101,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.112,0.224,0.224,0.352,0.361,0.360,0.360,0.233,0.111,0.102,0.237,0.237,0.338,0.337,0.302,0.302,0.201,0.101,0.000,0.133,0.237,0.343,0.343,0.210,0.105,0.000,0.000,0.105,0.209,0.323,0.514,0.596,0.613,0.746,0.681,0.493,0.372,0.126,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.135,0.301,0.301,0.301,0.165,0.000,0.000,0.102,0.203,0.203,0.318,0.216,0.115,0.115,0.000,0.000,0.102,0.102,0.102,0.102,0.101,0.291,0.393,0.505,0.515,0.325,0.331,0.219,0.107,0.107,0.000,0.139,0.139,0.139,0.247,0.109,0.218,0.345,0.236,0.236,0.127,0.000
1_the_of_and_information,0.191,0.379,0.579,0.955,1.135,1.191,1.175,0.984,0.729,0.632,0.448,0.263,0.147,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.103,0.103,0.103,0.103,0.000,0.000,0.000,0.000,0.131,0.296,0.296,0.400,0.372,0.311,0.311,0.207,0.104,0.000,0.129,0.129,0.129,0.129,0.000,0.000,0.000,0.000,0.137,0.276,0.430,0.613,0.655,0.631,0.717,0.673,0.494,0.378,0.139,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.134,0.298,0.298,0.298,0.164,0.000,0.000,0.000,0.000,0.000,0.117,0.117,0.117,0.117,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.116,0.116,0.116,0.116,0.000,0.110,0.110,0.110,0.110,0.000,0.105,0.105,0.105,0.212,0.107,0.218,0.376,0.269,0.269,0.158,0.000
2_the_of_and_to,0.192,0.391,0.600,0.913,1.022,1.035,0.964,0.772,0.470,0.399,0.380,0.260,0.260,0.120,0.000,0.113,0.227,0.355,0.355,0.354,0.371,0.351,0.458,0.346,0.319,0.211,0.219,0.359,0.369,0.483,0.501,0.523,0.613,0.703,0.731,0.683,0.480,0.276,0.114,0.000,0.000,0.000,0.105,0.261,0.429,0.574,0.599,0.442,0.275,0.130,0.000,0.000,0.116,0.116,0.235,0.406,0.451,0.618,0.672,0.618,0.458,0.400,0.227,0.232,0.383,0.274,0.398,0.398,0.370,0.491,0.493,0.490,0.489,0.513,0.388,0.373,0.360,0.328,0.477,0.370,0.367,0.391,0.377,0.547,0.676,0.772,0.836,0.918,0.836,0.603,0.514,0.262,0.110,0.110,0.000,0.000,0.000,0.000,0.000,0.128,0.338,0.468,0.583,0.566,0.471,0.459,0.510,0.565,0.610,0.727,0.775,0.761,0.732,0.602,0.489,0.497,0.524,0.603,0.641,0.618,0.617,0.434,0.406,0.371,0.343,0.527,0.571,0.626,0.613,0.429,0.410,0.362,0.244,0.374,0.363,0.380,0.513,0.383,0.258,0.133,0.000
3_the_of_to_methods,0.175,0.345,0.517,0.765,0.803,0.833,0.770,0.625,0.525,0.464,0.475,0.372,0.258,0.120,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.128,0.

In [ ]:
df

,x,y,Topic,Length
1,7.515896,-2.087861,0,2207
2,4.932786,4.860031,7,1482
7,8.899693,7.014271,4,2104
10,6.284402,-2.960776,0,880
12,3.841189,6.060935,8,1237
...,...,...,...,...
19636,8.261065,9.692606,12,450
19637,4.013752,6.058924,8,241
19638,7.984281,6.750187,11,1588
19641,5.679514,-0.058838,0,482


In [ ]:
topics_documents = topic_model.get_document_info(abstracts)

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(abstracts)
vis2 = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics, custom_labels=True)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics, custom_labels=True)

100%|██████████| 5/5 [00:00<00:00, 188.20it/s]


Each level represents a grouping of documents and topics at a specific level of detail or abstraction. As you move from level 0 to level 9, you are exploring the topics at different levels of granularity.



topic_model.set_topic_labels({1: "Space Travel", 7: "Religion"})

topic_model.visualize_topics_over_time(topics_over_time, topics=[9, 10, 72, 83, 87, 91])

In [ ]:
similar_topics, similarity = topic_model.find_topics("Transformers", top_n=5)
top_topics = []
for topic in similar_topics:
  top_topics.append(topic_model.get_topic(topic))
top_topics


[[('and', 0.0756500620982752),
  ('the', 0.056957215369176056),
  ('information', 0.05405192843466668),
  ('security', 0.05280541091324998),
  ('of', 0.052647991455166986),
  ('in', 0.04685306347042062),
  ('are', 0.03860437064685107),
  ('computer', 0.03649247255059024),
  ('other', 0.03595872784238487),
  ('systems', 0.035342366149218894)],
 [('the', 0.07307964818764212),
  ('of', 0.06381512641847745),
  ('and', 0.058264077301786926),
  ('to', 0.05390193632931076),
  ('encryption', 0.05065909236203995),
  ('security', 0.048292309404200824),
  ('key', 0.045030304321813285),
  ('is', 0.04033322010255342),
  ('system', 0.03896279655000349),
  ('are', 0.038484625916255975)],
 [('the', 0.09080818269551702),
  ('system', 0.07188872768693504),
  ('security', 0.06883818999572032),
  ('infosec', 0.06617852780857927),
  ('of', 0.05408404929875912),
  ('and', 0.052372166956596806),
  ('advanced', 0.04727037700612805),
  ('avionic', 0.042010513180793574),
  ('comsec', 0.042010513180793574),
  ('

In [ ]:
topics_documents

,Document,Topic,Name,CustomName,Representation,KeyBERT,Mistral,MMR,Representative_Docs,Top_n_words,Probability,Representative_document
0,"In retail and international banking, there is ...",1,1_the_of_and_information,Information Security and Local Area Networks,"[the, of, and, information, security, in, is, ...","[hackers, security, technical, industry, syste...","[Information Security and Local Area Networks,...","[the, of, and, information, security, in, is, ...",[Local area networks which handle many signals...,the - of - and - information - security - in -...,1.000000,False
1,"In the future, the security of computer and da...",2,2_the_of_and_to,Encryption and Security in Computer Systems,"[the, of, and, to, encryption, security, key, ...","[encryption, security, decryption, secure, sys...","[Encryption and Security in Computer Systems, ...","[the, of, and, to, encryption, security, key, ...",[This paper presents three important - and lik...,the - of - and - to - encryption - security - ...,1.000000,True
2,A computer system that has been developed for ...,1,1_the_of_and_information,Information Security and Local Area Networks,"[the, of, and, information, security, in, is, ...","[hackers, security, technical, industry, syste...","[Information Security and Local Area Networks,...","[the, of, and, information, security, in, is, ...",[Local area networks which handle many signals...,the - of - and - information - security - in -...,0.854016,False
3,The authors introduce the security compliance ...,4,4_the_system_security_infosec,Advanced avionics and information security,"[the, system, security, infosec, of, and, adva...","[avionics, avionic, infosec, comsec, systems, ...","[Advanced avionics and information security, ,...","[the, system, security, infosec, of, and, adva...",[The authors identify INFOSEC (information sec...,the - system - security - infosec - of - and -...,0.825683,True
4,This conference proceedings contains 43 papers...,-1,-1_and_the_information_security,Information systems and security,"[and, the, information, security, of, in, are,...","[systems, communications, technology, system, ...","[Information systems and security, , , , , , ,...","[and, the, information, security, of, in, are,...",[The following topics are dealt with: network ...,and - the - information - security - of - in -...,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...
73,The special characteristics of a small informa...,0,0_the_and_of_information,Information Security Assessment Methods,"[the, and, of, information, security, to, is, ...","[security, management, systems, technical, sys...","[Information Security Assessment Methods, , , ...","[the, and, of, information, security, to, is, ...",[Even if an organization has the best technica...,the - and - of - information - security - to -...,0.696436,False
74,Information security - a broader term than com...,-1,-1_and_the_information_security,Information systems and security,"[and, the, information, security, of, in, are,...","[systems, communications, technology, system, ...","[Information systems and security, , , , , , ,...","[and, the, information, security, of, in, are,...",[The following topics are dealt with: network ...,and - the - information - security - of - in -...,0.000000,True
75,"Recently, computer security and incidents of c...",0,0_the_and_of_information,Information Security Assessment Methods,"[the, and, of, information, security, to, is, ...","[security, management, systems, technical, sys...","[Information Security Assessment Methods, , , ...","[the, and, of, information, security, to, is, ...",[Even if an organization has the best technica...,the - and - of - information - security - to -...,0.776340,True
76,The following topics are dealt with: network c...,-1,-1_and_the_information_security,Information systems and security,"[and, the, information, security, of, in, are,...","[systems, communications, technology, system, ...","[Information systems and securi

In [ ]:
topic_model.visualize_heatmap(top_n_topics=50, custom_labels=False, title='<b>Similarity Matrix</b>',
                              n_clusters=5,
                              width=1000, height=1000)

In [ ]:
fig = topic_model.visualize_heatmap(top_n_topics=50, custom_labels=True, title='<b>Similarity Matrix</b>',
                              n_clusters=30,
                              width=1000, height=1000)
fig.write_html(path_file + "Similarity_Matrix.html")

In [ ]:
vis3 = topic_model.visualize_topics()
#vis3.write_html(path_file + "IntertopicDistanceMap.html")
topic_model.visualize_topics()

In [ ]:
import plotly.io as pio

In [ ]:
pio.write_html(visualization, path_file + "visualization.html")

In [ ]:
pio.write_html(vis2, path_file + "visualization2.html")

In [ ]:
pio.write_html(vis3, path_file + "visualization3.html")

## Multimodal data

You can convert images into sentences and then determine which topic of the model they belong to.
https://maartengr.github.io/BERTopic/getting_started/tips_and_tricks/tips_and_tricks.html#finding-similar-topics-between-models

In [ ]:
topic_model.topic_embeddings_

array([[-0.00174072,  0.00544258,  0.00102158, ...,  0.0093399 ,
        -0.01147885,  0.02099998],
       [-0.00874138,  0.01220342, -0.00545494, ...,  0.00935201,
         0.00242328,  0.01060997],
       [-0.00534367,  0.00409249,  0.00464408, ...,  0.01135131,
        -0.01176645,  0.01925318],
       ...,
       [-0.00074439, -0.00205662,  0.01908344, ...,  0.0064348 ,
        -0.01335396,  0.0177061 ],
       [-0.00915908,  0.01529152, -0.01227046, ...,  0.00304455,
        -0.0343275 ,  0.01975444],
       [ 0.00485607, -0.00689857,  0.01862592, ...,  0.00152195,
         0.01105705,  0.00300332]], dtype=float32)

In [ ]:
info_df = topic_model.get_topic_info()

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,CustomName,Representation,KeyBERT,SOLAR,MMR,Representative_Docs
0,-1,18956,-1_the_of_and_to,Cybersecurity Challenges in Digital Age,"[the, of, and, to, in, is, security, this, inf...","[cybersecurity, cyber, security, threats, atta...","[Cybersecurity Challenges in Digital Age, , , ...","[the, of, and, to, in, is, security, this, inf...",[Cybersecurity is a computing-based discipline...
1,0,5499,0_power_the_systems_and,Cybersecurity and Control Systems in Smart Grids,"[power, the, systems, and, control, of, grid, ...","[scada, cyber, cybersecurity, ieee, ics, syste...",[Cybersecurity and Control Systems in Smart Gr...,"[power, the, systems, and, control, of, grid, ...",[With the high penetration of distributed ener...
2,1,2902,1_security_information_management_risk,Information Security Management and Business R...,"[security, information, management, risk, the,...","[methodology, implementation, management, secu...",[Information Security Management and Business ...,"[security, information, management, risk, the,...",[The Information security is considered a risk...
3,2,2644,2_detection_intrusion_network_the,Network Intrusion Detection using Machine Lear...,"[detection, intrusion, network, the, ids, to, ...","[intrusion, intrusions, classification, detect...",[Network Intrusion Detection using Machine Lea...,"[detection, intrusion, network, the, ids, to, ...",[Network intrusion detection is a key step in ...
4,3,2398,3_students_cybersecurity_education_and,Enhancing K-12 Students' Cybersecurity Educati...,"[students, cybersecurity, education, and, to, ...","[acm, ieee, cybersecurity, computing, cyber, u...",[Enhancing K-12 Students' Cybersecurity Educat...,"[students, cybersecurity, education, and, to, ...",[Cybersecurity is important for many applicati...
...,...,...,...,...,...,...,...,...,...
58,57,154,57_cities_smart_city_and,Smart City Technologies and Urban Development,"[cities, smart, city, and, urban, technologies...","[iot, security, cybersecurity, ict, technologi...",[Smart City Technologies and Urban Development...,"[cities, smart, city, and, urban, technologies...",[Internet of Things is providing concept of co...
59,58,151,58_flow_policies_we_language,Access Control and Information Security Policies,"[flow, policies, we, language, information, no...","[protocols, protocol, cryptographic, proofs, s...",[Access Control and Information Security Polic...,"[flow, policies, we, language, information, no...",[Access control is an information security pro...
60,59,148,59_trusted_trust_computing_tpm,Trusted Computing and Security Model,"[trusted, trust, computing, tpm, the, model, p...","[trusted, computing, secure, trust, credibilit...","[Trusted Computing and Security Model, , , , ,...","[trusted, trust, computing, tpm, the, model, p...",[The trusted operation is the key factor for t...
61,60,146,60_payment_commerce_online_mobile,Consumer Trust in Online Payment Systems,"[payment, commerce, online, mobile, trust, stu...","[consumers, fintech, consumer, cybersecurity, ...","[Consumer Trust in Online Payment Systems, , ,...","[payment, commerce, online, mobile, trust, stu...",[The ever-increasing acceptance of cryptocurre...


## Save list of topics, list of documents, position in space

In [ ]:
total_count = info_df['Count'].sum()

info_df['percentage'] = (info_df['Count'] / total_count)
info_df

,Topic,Count,Name,CustomName,Representation,KeyBERT,SOLAR,MMR,Representative_Docs,percentage
0,-1,18956,-1_the_of_and_to,Cybersecurity Challenges in Digital Age,"[the, of, and, to, in, is, security, this, inf...","[cybersecurity, cyber, security, threats, atta...","[Cybersecurity Challenges in Digital Age, , , ...","[the, of, and, to, in, is, security, this, inf...",[Cybersecurity is a computing-based discipline...,0.304055
1,0,5499,0_power_the_systems_and,Cybersecurity and Control Systems in Smart Grids,"[power, the, systems, and, control, of, grid, ...","[scada, cyber, cybersecurity, ieee, ics, syste...",[Cybersecurity and Control Systems in Smart Gr...,"[power, the, systems, and, control, of, grid, ...",[With the high penetration of distributed ener...,0.088204
2,1,2902,1_security_information_management_risk,Information Security Management and Business R...,"[security, information, management, risk, the,...","[methodology, implementation, management, secu...",[Information Security Management and Business ...,"[security, information, management, risk, the,...",[The Information security is considered a risk...,0.046548
3,2,2644,2_detection_intrusion_network_the,Network Intrusion Detection using Machine Lear...,"[detection, intrusion, network, the, ids, to, ...","[intrusion, intrusions, classification, detect...",[Network Intrusion Detection using Machine Lea...,"[detection, intrusion, network, the, ids, to, ...",[Network intrusion detection is a key step in ...,0.042410
4,3,2398,3_students_cybersecurity_education_and,Enhancing K-12 Students' Cybersecurity Educati...,"[students, cybersecurity, education, and, to, ...","[acm, ieee, cybersecurity, computing, cyber, u...",[Enhancing K-12 Students' Cybersecurity Educat...,"[students, cybersecurity, education, and, to, ...",[Cybersecurity is important for many applicati...,0.038464
...,...,...,...,...,...,...,...,...,...,...
58,57,154,57_cities_smart_city_and,Smart City Technologies and Urban Development,"[cities, smart, city, and, urban, technologies...","[iot, security, cybersecurity, ict, technologi...",[Smart City Technologies and Urban Development...,"[cities, smart, city, and, urban, technologies...",[Internet of Things is providing concept of co...,0.002470
59,58,151,58_flow_policies_we_language,Access Control and Information Security Policies,"[flow, policies, we, language, information, no...","[protocols, protocol, cryptographic, proofs, s...",[Access Control and Information Security Polic...,"[flow, policies, we, language, information, no...",[Access control is an information security pro...,0.002422
60,59,148,59_trusted_trust_computing_tpm,Trusted Computing and Security Model,"[trusted, trust, computing, tpm, the, model, p...","[trusted, computing, secure, trust, credibilit...","[Trusted Computing and Security Model, , , , ,...","[trusted, trust, computing, tpm, the, model, p...",[The trusted operation is the key factor for t...,0.002374
61,60,146,60_payment_commerce_online_mobile,Consumer Trust in Online Payment Systems,"[payment, commerce, online, mobile, trust, stu...","[consumers, fintech, consumer, cybersecurity, ...","[Consumer Trust in Online Payment Systems, , ,...","[payment, commerce, online, mobile, trust, stu...",[The ever-increasing acceptance of cryptocurre...,0.002342


In [ ]:
info_df.to_csv('info_model_bertopic_solar.csv', index=False)
topics_documents.to_csv('document_topics_abstracts.csv', index=False)
df.to_csv(path_file + 'espacio.csv', index=False)


## Visualization over time

In [ ]:
timestamps = pd.to_datetime(dataset.Year, format='%Y', errors='coerce').to_list()
abs = dataset.Abstract.to_list()

# Create topics over time
model2 = BERTopic(verbose=True)
topics, probs = model2.fit_transform(abs)
topics_over_time = model2.topics_over_time(abs, timestamps)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1949 [00:00<?, ?it/s]

2024-01-21 18:30:42,777 - BERTopic - Transformed documents to Embeddings
2024-01-21 18:31:22,470 - BERTopic - Reduced dimensionality
2024-01-21 18:31:27,744 - BERTopic - Clustered reduced embeddings
51it [01:18,  1.53s/it]


In [ ]:
vis4 = model2.visualize_topics_over_time(topics_over_time, topics=range(300), custom_labels=True)
pio.write_html(vis4, path_file + "visualization4.html")
model2.visualize_topics_over_time(topics_over_time, topics=range(300), custom_labels=True)

In [ ]:
df_subtopic = topics_documents

In [ ]:
merge_df = pd.concat([dataset, df_subtopic], axis=1)

In [ ]:
merge_df.columns

Index(['Authors', 'Author full names', 'Author(s) ID', 'Title', 'Year',
       'Source title', 'Volume', 'Issue', 'Art. No.', 'Page start', 'Page end',
       'Page count', 'Cited by', 'DOI', 'Link', 'Affiliations',
       'Authors with affiliations', 'Abstract', 'Author Keywords',
       'Index Keywords', 'Molecular Sequence Numbers', 'Chemicals/CAS',
       'Tradenames', 'Manufacturers', 'Funding Details', 'Funding Texts',
       'References', 'Correspondence Address', 'Editors', 'Publisher',
       'Sponsors', 'Conference name', 'Conference date', 'Conference location',
       'Conference code', 'ISSN', 'ISBN', 'CODEN', 'PubMed ID',
       'Language of Original Document', 'Abbreviated Source Title',
       'Document Type', 'Publication Stage', 'Open Access', 'Source', 'EID',
       'Document', 'Topic', 'Name', 'CustomName', 'Representation', 'KeyBERT',
       'SOLAR', 'MMR', 'Representative_Docs', 'Top_n_words', 'Probability',
       'Representative_document'],
      dtype='object')

## Relevance Index

The metric that sorts papers by order of relevance:
\[
\text{Relevance Index} = \frac{\text{Cited by}}{\text{Years since publication}}
\]

- **Cited by:** This component measures the influence and recognition a paper has received from the academic community. The more citations it has, the higher this value, indicating greater visibility and perceived relevance by other researchers.

- **Years since publication:** This component represents the time that has elapsed since the paper was published. The idea is that over time, the relevance of a paper can decrease as new advancements and discoveries may have been made.

- **Relevance Index:** The metric itself is a ratio between the citations received and the time elapsed since publication. A higher Relevance Index indicates that the paper has maintained or even increased its significance over time compared to other papers.

**Examples:**

1. **Paper A:** Published 5 years ago, has received 100 citations.
$$
    \text{Relevance Index}_A = \frac{100}{5} = 20
$$

2. **Paper B:** Published 2 years ago, has received 50 citations.
$$
    \text{Relevance Index}_B = \frac{50}{2} = 25
$$


In [ ]:
import datetime

current_year = datetime.datetime.now().year

# Calcular la diferencia entre el año actual y el año de publicación
merge_df['Years_since_publication'] = current_year - merge_df['Year']

# Calcular el índice de relevancia
merge_df['Relevance_Index'] = merge_df['Cited by'] / merge_df['Years_since_publication'].replace(0, 1)


In [ ]:
merge_df[['Title', 'Year', 'Cited by', 'Years_since_publication', 'Relevance_Index']]

,Title,Year,Cited by,Years_since_publication,Relevance_Index
0,Numerical Simulation of Free Surface Water Wav...,2024,0,-1,-0.000000
1,Medical Image Processing based on Generative A...,2024,0,-1,-0.000000
2,Predicting Election Results with Machine Learn...,2024,0,-1,-0.000000
3,On the interest of artificial intelligence app...,2024,0,-1,-0.000000
4,Leveraging machine learning for automatic topi...,2024,0,-1,-0.000000
...,...,...,...,...,...
19638,Human Reliability Engineering,1978,10,45,0.222222
19639,Mechanisation of Serials Records: A Literature...,1970,1,53,0.018868
19640,LITERATURE REVIEW ISSUE.,1975,0,48,0.000000
19641,Comprehensive hospital computer applications p...,1975,0,48,0.000000


In [ ]:
slr_df = merge_df[merge_df["Topic"]==14].sort_values(by="Relevance_Index", ascending = False)
slr_df = slr_df[['Title', 'Year', 'Cited by', 'Abstract', 'Author Keywords','Index Keywords' , 'Document Type', 'Probability', 'Representative_document', 'Relevance_Index']]

In [ ]:
slr_df.to_excel("subtopic14_SLR.xlsx")